## Data uploading

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten,
                                     Activation, Dense, Dropout,
                                     BatchNormalization)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
import os
import shutil
import itertools
import pathlib
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

2024-04-14 17:47:56.319867: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-14 17:47:56.345828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 17:47:56.345855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 17:47:56.346518: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 17:47:56.351090: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import os
import pandas as pd

train_data_path = '../data/Training'

filepaths = []
labels = []

folds = [fold for fold in os.listdir(train_data_path)
         if os.path.isdir(os.path.join(train_data_path, fold))]

for fold in folds:
    f_path = os.path.join(train_data_path, fold)
    # Skip .DS_Store
    if not os.path.isdir(f_path):
        continue
    filelists = os.listdir(f_path)
    for file in filelists:
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='label')
train_df = pd.concat([Fseries, Lseries], axis=1)
train_df

,filepaths,label
0,../data/Training/pituitary/Tr-pi_0634.jpg,pituitary
1,../data/Training/pituitary/Tr-pi_1202.jpg,pituitary
2,../data/Training/pituitary/Tr-pi_1120.jpg,pituitary
3,../data/Training/pituitary/Tr-pi_0367.jpg,pituitary
4,../data/Training/pituitary/Tr-pi_0801.jpg,pituitary
...,...,...
5707,../data/Training/glioma/Tr-gl_0372.jpg,glioma
5708,../data/Training/glioma/Tr-gl_0897.jpg,glioma
5709,../data/Training/glioma/Tr-gl_1206.jpg,glioma
5710,../data/Training/glioma/Tr-gl_0736.jpg,glioma


In [4]:
test_data_path = '../data/Testing'

filepaths = []
labels = []

folds = [fold for fold in os.listdir(test_data_path)
         if os.path.isdir(os.path.join(test_data_path, fold))]

for fold in folds:
    f_path = os.path.join(test_data_path, fold)
    # Skip .DS_Store
    if not os.path.isdir(f_path):
        continue
    filelists = os.listdir(f_path)
    for file in filelists:
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='label')
test_df = pd.concat([Fseries, Lseries], axis=1)
test_df

,filepaths,label
0,../data/Testing/pituitary/Te-pi_0272.jpg,pituitary
1,../data/Testing/pituitary/Te-pi_0243.jpg,pituitary
2,../data/Testing/pituitary/Te-pi_0061.jpg,pituitary
3,../data/Testing/pituitary/Te-pi_0262.jpg,pituitary
4,../data/Testing/pituitary/Te-pi_0224.jpg,pituitary
...,...,...
1306,../data/Testing/glioma/Te-gl_0122.jpg,glioma
1307,../data/Testing/glioma/Te-gl_0092.jpg,glioma
1308,../data/Testing/glioma/Te-gl_0238.jpg,glioma
1309,../data/Testing/glioma/Te-gl_0098.jpg,glioma


## TensorFlow Image Data Generator and classes dataframes

In [5]:
img_size = (224, 244)
batch_size = 16
valid, test = train_test_split(test_df, train_size=0.5,
                               shuffle=True, random_state=42)
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df, x_col='filepaths',
                                       y_col='label', target_size=img_size,
                                       class_mode='categorical',
                                       color_mode='rgb', shuffle=True,
                                       batch_size=batch_size)

valid_gen = ts_gen.flow_from_dataframe(valid, x_col='filepaths',
                                       y_col='label', target_size=img_size,
                                       class_mode='categorical',
                                       color_mode='rgb', shuffle=True,
                                       batch_size=batch_size)

test_gen = ts_gen.flow_from_dataframe(test, x_col='filepaths', y_col='label',
                                      target_size=img_size,
                                      class_mode='categorical',
                                      color_mode='rgb',
                                      shuffle=False, batch_size=batch_size)
# Classes dataframes
gen_dict = train_gen.class_indices
classes = list(gen_dict.keys())
images, labels = next(train_gen)

Found 5712 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.


## Model Structure

In [6]:
img_shape = (img_size[0], img_size[1], 3)
num_class = len(classes)

base_model = tf.keras.applications.efficientnet.EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=img_shape,
    pooling='max')
model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=regularizers.l2(l=0.016),
          activity_regularizer=regularizers.l1(0.007),
          bias_regularizer=regularizers.l1(0.007), activation='relu'),
    Dropout(rate=0.4, seed=75),
    Dense(num_class, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy',
              metrics=['accuracy'])

2024-04-14 17:47:58.196514: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-14 17:47:58.222412: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Model main info

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional  (None, 1536)              10783535  
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 1536)              6144      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 256)               393472    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 11184179 (42.66 MB)
Trainable params: 1109

## Iterative training

In [8]:
Epochs = 2

history = model.fit(x=train_gen, epochs=Epochs,
                    verbose=1, validation_data=valid_gen,
                    validation_steps=None, shuffle=False)

Epoch 1/2
357/357 [==============================] - 906s 2s/step - loss: 5.8129 - accuracy: 0.8713 - val_loss: 3.8383 - val_accuracy: 0.9435
Epoch 2/2
357/357 [==============================] - 873s 2s/step - loss: 2.7145 - accuracy: 0.9631 - val_loss: 1.8220 - val_accuracy: 0.9908


In [11]:
# Load the single image
image = tf.keras.utils.load_img(
    '../data/Testing/notumor/Te-no_0404.jpg',
    target_size=(img_size[0], img_size[1]))
image = tf.keras.utils.img_to_array(image)
image = tf.expand_dims(image, axis=0)
image = tf.keras.applications.efficientnet.preprocess_input(image)
predictions = model.predict(image)
predicted_class = tf.math.argmax(predictions, axis=1)
class_dict_local = {0: 'glioma', 1: 'meningioma', 2: 'notumor', 3: 'pituitary'}
predicted_class_literal = class_dict_local[predicted_class.numpy()[0]]
print('The predicted class is:', predicted_class_literal)

1/1 [==============================] - 1s 1s/step
The predicted class is: notumor


## Common Metrics

In [12]:
train_score = model.evaluate(train_gen, steps=16, verbose=1)
valid_score = model.evaluate(valid_gen, steps=16, verbose=1)
test_score = model.evaluate(test_gen, steps=16, verbose=1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

16/16 [==============================] - 7s 459ms/step - loss: 1.8266 - accuracy: 0.9883
Train Loss:  1.7999218702316284
Train Accuracy:  0.99609375
Validation Loss:  1.8259507417678833
Validation Accuracy:  0.98828125
Test Loss:  1.8265697956085205
Test Accuracy:  0.98828125


## Metrics of classes

In [13]:
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds, axis=1)
print(classification_report(test_gen.classes, y_pred, target_names=classes))

/tmp/ipykernel_22477/675270116.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(test_gen)


              precision    recall  f1-score   support

      glioma       0.99      0.97      0.98       149
  meningioma       0.97      0.94      0.96       148
     notumor       0.99      1.00      0.99       208
   pituitary       0.97      1.00      0.98       151

    accuracy                           0.98       656
   macro avg       0.98      0.98      0.98       656
weighted avg       0.98      0.98      0.98       656

